In [0]:
FLUXSTD_FILE = f'/datascope/subaru/data/targeting/dSph/draco/PS1_Dra_fluxstd_3_dobos.csv'
SKY_FILE = f'/datascope/subaru/data/targeting/dSph/draco/sky_draco.feather'
OBS_PATH = f'/datascope/subaru/data/targeting/dSph/draco/draco_tpall3e_g24.cat'

GALAXY = 'draco'

 ```sql
Run this ON HLSP_PS1_PSC
-----

DECLARE @ra AS float = 227.29725;
DECLARE @dec AS float = 67.21436111;
DECLARE @rad AS float = 120; 

IF OBJECT_ID('PS1_UMI_fluxstd_1') IS NOT NULL DROP TABLE PS1_UMI_fluxstd_1

SELECT --TOP 10
    p.objID as obj_id,
    p.raMean AS RA, p.decMean AS Dec,
    gPSFMag, gPSFMagErr, rPSFMag, rPSFMagErr, iPSFMag, iPSFMagErr, zPSFMag, zPSFMagErr, yPSFMag, yPSFMagErr
INTO Mydb.PS1_UMI_fluxstd_1
FROM pointsource_magnitudes_view AS p
INNER JOIN fGetNearbyObjEq(@ra, @dec, @rad) AS r ON p.objid=r.objid
WHERE p.ps_score > 0.9
    AND gPSFMag != -999 AND rPSFMag != -999 AND zPSFMag != -999
    AND gPSFMag BETWEEN 13 AND 19
    AND gPSFMag - rPSFMag BETWEEN 0.25 AND 0.4

CREATE CLUSTERED INDEX IX_PS1_UMI_fluxstd_1 ON PS1_UMI_fluxstd_1 (obj_id)


----
obj_id like 186262271624076579



Run this on GAIA_DR2
----------------------

SELECT 
    m.obj_id, g.source_id,
    g.ra, g.dec,
    g.parallax, g.parallax_error,
    g.pmra, g.pmra_error, g.pmdec, g.pmdec_error,
    g.radial_velocity, g.radial_velocity_error,
    n.angular_distance,
    gPSFMag, gPSFMagErr, rPSFMag, rPSFMagErr, iPSFMag, iPSFMagErr, zPSFMag, zPSFMagErr, yPSFMag, yPSFMagErr
INTO PS1_UMI_fluxstd_2
FROM MYDB.PS1_UMI_fluxstd_1 m 
INNER JOIN panstarrs1_best_neighbour n
    ON m.obj_id = n.clean_panstarrs1_oid
INNER JOIN gaia_source g
    ON g.source_id = n.source_id
````

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle, Distance

In [0]:
# if 'debug' not in globals():
#     import debugpy
#     debugpy.listen(('0.0.0.0', 5698))
#     debug = True

In [0]:
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color

In [0]:
from pfs.ga.targeting.io import PfsFluxStdReader, PfsSkyReader
from pfs.ga.targeting.data import Observation

In [0]:
fluxstd = pd.read_csv(FLUXSTD_FILE)
fluxstd.rename(columns={ 'ra': 'RA', 'dec': 'Dec' }, inplace=True)

fluxstd = Observation(data=fluxstd)

fluxstd.data

In [0]:
fluxstd.data.columns

In [0]:
# Check duplicates

duplicates = fluxstd.data.sort_values(['obj_id', 'angular_distance']).duplicated('obj_id', keep='first')
duplicates.sum()

In [0]:
fluxstd = fluxstd.filter(mask=~duplicates)

In [0]:
fluxstd.shape

In [0]:
dm_mask = fluxstd.data['parallax'] > 0
DM = Distance(parallax=fluxstd.data['parallax'][dm_mask].values * u.mas).distmod.value

In [0]:
plt.hist(fluxstd.data['pmra'], bins=np.linspace(-30, 30, 100))
plt.xlabel('pmra')

In [0]:
plt.hist(fluxstd.data['pmdec'], bins=np.linspace(-30, 30, 100))
plt.xlabel('pmdec')

In [0]:
from pfs.ga.targeting.targets.dsph import GALAXIES
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.instrument import SubaruWFC, SubaruPFI
from pfs.ga.targeting.diagram import FOV, FP

In [0]:
target = GALAXIES[GALAXY]

In [0]:
obs = target.get_text_observation_reader().read(OBS_PATH)

In [0]:
pointing = target.get_pointings(SubaruPFI)[0]
print(pointing.ra, pointing.dec, pointing.posang)

wcs = WcsProjection(pointing, proj='TAN')
wfc = SubaruWFC(wcs.pointing)
pfi = SubaruPFI(projection=wfc, instrument_options={ 'layout': 'calibration' })
fov = FOV(projection=wcs)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(8, 8), dpi=120, subplot_kw=dict(projection=fov.projection.wcs))

fov.plot_catalog(ax, obs, color='gray')
# fov.plot_catalog(ax, sky, color='b', alpha=0.3)
fov.plot_catalog(ax, fluxstd, mask=dm_mask, color='r', size=5)

# ax.plot(target.pos.ra, target.pos.dec, marker='x', markersize=30, color='r', transform=ax.get_transform('world'))

pfi.plot_focal_plane(ax, fov, color='red', lw=1, corners=True)

ax.set_xlim(2, -2.5)
ax.set_ylim(-2, 1)

ax.grid()

In [0]:
plt.plot(fluxstd.data['gPSFMag'] - fluxstd.data['rPSFMag'], fluxstd.data['gPSFMag'], '.')
plt.gca().invert_yaxis()
plt.xlabel('PS1 g - r')
plt.ylabel('PS1 g')

In [0]:
plt.plot(fluxstd.data['gPSFMag'][dm_mask] - fluxstd.data['rPSFMag'][dm_mask],
         fluxstd.data['gPSFMag'][dm_mask] - DM, '.')


px_mask = (fluxstd.data['parallax'][dm_mask] / fluxstd.data['parallax_error'][dm_mask]) > 10

plt.plot(fluxstd.data['gPSFMag'][dm_mask][px_mask] - fluxstd.data['rPSFMag'][dm_mask][px_mask],
         fluxstd.data['gPSFMag'][dm_mask][px_mask] - DM[px_mask], '.')

plt.gca().invert_yaxis()
plt.xlabel('PS1 g - r')
plt.ylabel('abs PS1 g')

In [0]:
DM

In [0]:
dm_mask.sum()

In [0]:
# Write out filtered list to a feather file
dir = os.path.dirname(FLUXSTD_FILE)
fn, _ = os.path.splitext(os.path.basename(FLUXSTD_FILE))
print(dir, fn)

path = os.path.join(dir, fn + '.feather')
fluxstd.data.to_feather(path)

path

In [0]:
fluxstd.data.columns